# 함수 선언

하나의 함수로 만들어 연도 범위 지정하고 데이터 반환 받기

In [1]:
import os
from mysql import connector
from dotenv import load_dotenv
import pandas as pd
import datetime

In [2]:
def load_data(start_year, end_year):
    # 환경변수 로드
    load_dotenv()
    
    NAME = os.getenv("DB_NAME")
    USER = os.getenv("DB_USER")
    PASSWORD = os.getenv("DB_PASSWORD")
    IP = os.getenv("DB_IP")
    PORT = os.getenv("DB_PORT")
    
    class Database:
        _connection = None  # 클래스 변수로 연결 상태 관리
    
        @staticmethod
        def get_connection():
            if Database._connection is None or not Database._connection.is_connected():
                Database._connection = connector.connect(
                    database = NAME,
                    user = USER,
                    password = PASSWORD,
                    host = IP,
                    port = PORT
                )
            return Database._connection
    
    try:
        conn = Database.get_connection()
        cursor = conn.cursor()
        years_range = range(start_year, end_year + 1)
        rc_sum_str = " + ".join([f"SUM(rc.`{year}`)" for year in years_range])
        irc_sum_str = " + ".join([f"SUM(irc.`{year}`)" for year in years_range])
    
        cursor.execute(f"""
            SELECT 
                book.ID,
                registration_year,
                registration_month,
                get_course,
                DDC,
                title,
                publication_year,
                location,
                duration,
                COALESCE({rc_sum_str}, 0) AS total_rent,
                COALESCE({irc_sum_str}, 0) AS rent_count,
                ID_count
            FROM book
            LEFT JOIN recent_rent       ON book.ID = recent_rent.ID
            LEFT JOIN rent_count AS rc        ON book.ID = rc.ID
            LEFT JOIN ISBN_rent_count AS irc   ON book.ISBN = irc.ISBN
            GROUP BY book.ID, registration_year, registration_month,
                     get_course, DDC, publication_year, location, duration
            ORDER BY book.ID
        """)
        rows_base = cursor.fetchall()
        cursor.close()
    
    except connector.Error as e:
        print(f"Error: {e}")
    
    df = pd.DataFrame(rows_base, columns=[
            'ID', '등록연도', '등록월', '수서방법',
            '분류코드', '제목', '출판연도', '소장위치', '최근대출',
            '총 대출 횟수', 'rent_count', 'book_count'
        ])

    try:
        conn = Database.get_connection()
        cursor = conn.cursor()
        years_range = range(start_year, end_year + 1)
        sum_str = " + ".join([f"SUM(`{year}`)" for year in years_range])
        cursor.execute(f"SELECT title, ID_count, COALESCE({sum_str}, 0) AS rent_count FROM None_ISBN_rent_count GROUP BY title;")
        rows = cursor.fetchall()
        cursor.close()
    except connector.Error as e:
        print(f"Error: {e}")
    
    None_ISBN_df = pd.DataFrame(rows, columns=['제목', 'book_count', 'rent_count'])
    
    None_ISBN_df = None_ISBN_df.astype(object)
    None_ISBN_df['rent_count'] = None_ISBN_df['rent_count'].astype(float)
    None_ISBN_df['book_count'] = None_ISBN_df['book_count'].astype(float)

    df_merged = pd.merge(df, None_ISBN_df, on='제목', how='left')
    df_merged['rent_count_x'] = df_merged['rent_count_x'].fillna(df_merged['rent_count_y'])
    df_merged['book_count_x'] = df_merged['book_count_x'].fillna(df_merged['book_count_y'])
    df_merged.drop('rent_count_y', axis=1, inplace=True)
    df_merged.drop('book_count_y', axis=1, inplace=True)
    df_merged.rename(columns={'rent_count_x': 'rent_count', 'book_count_x': 'book_count'}, inplace=True)
    df_merged.drop(columns=['제목'], inplace=True)
    df_merged['ID'] = df_merged['ID'].str.split('_').str[-1].astype(int)
    df_merged['수서방법'] = df_merged['수서방법'].astype('category')
    df_merged['분류코드'] = df_merged['분류코드'].astype(float)
    df_merged['출판연도'] = df_merged['출판연도'].astype(int)
    df_merged['소장위치'] = df_merged['소장위치'].astype('category')
    df_merged['최근대출'] = df_merged['최근대출'].fillna(7305).astype(int)
    df_merged['rent_count'] = df_merged['rent_count'].fillna(0).astype(int)
    df_merged['book_count'] = df_merged['book_count'].fillna(0).astype(int)
    df_merged['총 대출 횟수'] = df_merged['총 대출 횟수'].astype(int)
    df_onehot = pd.get_dummies(df_merged)

    end_date_str = "2024-10-31"
    start_date = datetime.date(start_year, 1, 1)           # 2019-01-01
    year_, month_, day_ = map(int, end_date_str.split('-'))  # 2024, 10, 31
    end_date = datetime.date(year_, month_, day_)
    diff_days = (end_date - start_date).days  # 2019-01-01 ~ 2024-10-31까지 일수
    df_onehot.loc[df_onehot['최근대출'] > diff_days, '최근대출'] = diff_days
    
    y_data = df_onehot['rent_count'] / df_onehot['book_count']
    y_data = y_data.fillna(0)
    df_onehot.drop(columns=['rent_count', 'book_count'], inplace=True)
    date_series = pd.to_datetime({
        "year":  df_onehot["등록연도"],
        "month": df_onehot["등록월"],
        "day":   1
    })
    end_date = pd.to_datetime("2024-10-31")
    df_onehot["reg_diff_days"] = (end_date - date_series).dt.days
    df_onehot["최근대출"] = df_onehot["최근대출"].fillna(df_onehot["reg_diff_days"])
    df_onehot.loc[df_onehot["최근대출"] > df_onehot["reg_diff_days"], "최근대출"] = df_onehot["reg_diff_days"]
    df_onehot.drop("reg_diff_days", axis=1, inplace=True)
    df_onehot["최근대출"] = df_onehot["최근대출"].astype(int)

    return df_onehot, y_data

# 데이터 불러오기

In [3]:
x_5, y_5 = load_data(2019, 2024)
x_10, y_10 = load_data(2014, 2024)
x_15, y_15 = load_data(2009, 2024)
x_20, y_20 = load_data(2004, 2024)

In [4]:
x_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299315 entries, 0 to 299314
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ID           299315 non-null  int64  
 1   등록연도         299315 non-null  int64  
 2   등록월          299315 non-null  int64  
 3   분류코드         299315 non-null  float64
 4   출판연도         299315 non-null  int64  
 5   최근대출         299315 non-null  int64  
 6   총 대출 횟수      299315 non-null  int64  
 7   수서방법_기타      299315 non-null  bool   
 8   수서방법_사서선정    299315 non-null  bool   
 9   수서방법_수서정보없음  299315 non-null  bool   
 10  수서방법_수업지정    299315 non-null  bool   
 11  수서방법_이용자희망   299315 non-null  bool   
 12  수서방법_학과신청    299315 non-null  bool   
 13  소장위치_4층인문    299315 non-null  bool   
 14  소장위치_보존서고    299315 non-null  bool   
dtypes: bool(8), float64(1), int64(6)
memory usage: 18.3 MB


In [5]:
x_10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299315 entries, 0 to 299314
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ID           299315 non-null  int64  
 1   등록연도         299315 non-null  int64  
 2   등록월          299315 non-null  int64  
 3   분류코드         299315 non-null  float64
 4   출판연도         299315 non-null  int64  
 5   최근대출         299315 non-null  int64  
 6   총 대출 횟수      299315 non-null  int64  
 7   수서방법_기타      299315 non-null  bool   
 8   수서방법_사서선정    299315 non-null  bool   
 9   수서방법_수서정보없음  299315 non-null  bool   
 10  수서방법_수업지정    299315 non-null  bool   
 11  수서방법_이용자희망   299315 non-null  bool   
 12  수서방법_학과신청    299315 non-null  bool   
 13  소장위치_4층인문    299315 non-null  bool   
 14  소장위치_보존서고    299315 non-null  bool   
dtypes: bool(8), float64(1), int64(6)
memory usage: 18.3 MB


In [6]:
x_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299315 entries, 0 to 299314
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ID           299315 non-null  int64  
 1   등록연도         299315 non-null  int64  
 2   등록월          299315 non-null  int64  
 3   분류코드         299315 non-null  float64
 4   출판연도         299315 non-null  int64  
 5   최근대출         299315 non-null  int64  
 6   총 대출 횟수      299315 non-null  int64  
 7   수서방법_기타      299315 non-null  bool   
 8   수서방법_사서선정    299315 non-null  bool   
 9   수서방법_수서정보없음  299315 non-null  bool   
 10  수서방법_수업지정    299315 non-null  bool   
 11  수서방법_이용자희망   299315 non-null  bool   
 12  수서방법_학과신청    299315 non-null  bool   
 13  소장위치_4층인문    299315 non-null  bool   
 14  소장위치_보존서고    299315 non-null  bool   
dtypes: bool(8), float64(1), int64(6)
memory usage: 18.3 MB


In [7]:
x_20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299315 entries, 0 to 299314
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   ID           299315 non-null  int64  
 1   등록연도         299315 non-null  int64  
 2   등록월          299315 non-null  int64  
 3   분류코드         299315 non-null  float64
 4   출판연도         299315 non-null  int64  
 5   최근대출         299315 non-null  int64  
 6   총 대출 횟수      299315 non-null  int64  
 7   수서방법_기타      299315 non-null  bool   
 8   수서방법_사서선정    299315 non-null  bool   
 9   수서방법_수서정보없음  299315 non-null  bool   
 10  수서방법_수업지정    299315 non-null  bool   
 11  수서방법_이용자희망   299315 non-null  bool   
 12  수서방법_학과신청    299315 non-null  bool   
 13  소장위치_4층인문    299315 non-null  bool   
 14  소장위치_보존서고    299315 non-null  bool   
dtypes: bool(8), float64(1), int64(6)
memory usage: 18.3 MB


# 파일 병합

In [8]:
# X 합치기
merge_data_x = pd.concat([x_20, x_15, x_10, x_5], axis=0, ignore_index=True)

# y 합치기
merge_data_y = pd.concat([y_20, y_15, y_10, y_5], axis=0, ignore_index=True)

In [9]:
merge_data_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197260 entries, 0 to 1197259
Data columns (total 15 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   ID           1197260 non-null  int64  
 1   등록연도         1197260 non-null  int64  
 2   등록월          1197260 non-null  int64  
 3   분류코드         1197260 non-null  float64
 4   출판연도         1197260 non-null  int64  
 5   최근대출         1197260 non-null  int64  
 6   총 대출 횟수      1197260 non-null  int64  
 7   수서방법_기타      1197260 non-null  bool   
 8   수서방법_사서선정    1197260 non-null  bool   
 9   수서방법_수서정보없음  1197260 non-null  bool   
 10  수서방법_수업지정    1197260 non-null  bool   
 11  수서방법_이용자희망   1197260 non-null  bool   
 12  수서방법_학과신청    1197260 non-null  bool   
 13  소장위치_4층인문    1197260 non-null  bool   
 14  소장위치_보존서고    1197260 non-null  bool   
dtypes: bool(8), float64(1), int64(6)
memory usage: 73.1 MB


In [10]:
merge_data_y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 1197260 entries, 0 to 1197259
Series name: None
Non-Null Count    Dtype  
--------------    -----  
1197260 non-null  float64
dtypes: float64(1)
memory usage: 9.1 MB


# XGBoost

In [11]:
# 모듈 불러오기
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [13]:
def xgb(n, d):
    # 모듈 불러오기
    import xgboost as xgb
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error, r2_score
    import numpy as np

    # 데이터 분리
    X_train, X_test, y_train, y_test = train_test_split(merge_data_x, merge_data_y, test_size=0.3, random_state=42)

    # 모델 생성
    xgb_reg = xgb.XGBRegressor(
        n_estimators = n,   # 트리 개수
        learning_rate = 0.05,  # 학습률
        max_depth = d,        # 트리 최대 깊이
        subsample = 0.8,
        colsample_bytree=0.8,
        random_state = 42
    )
    xgb_reg.fit(X_train, y_train)
    y_pred = xgb_reg.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    print(f"RMSE: {rmse:.4f}")
    print(f"R^2 : {r2:.4f}")

In [14]:
xgb(500, 7)

RMSE: 1.5274
R^2 : 0.8860


In [15]:
xgb(1000, 7)

RMSE: 1.4702
R^2 : 0.8944


In [16]:
xgb(1000, 10)

RMSE: 1.3308
R^2 : 0.9135


In [17]:
xgb(300, 7)

RMSE: 1.5691
R^2 : 0.8797


In [18]:
xgb(300, 5)

RMSE: 1.6614
R^2 : 0.8651


In [19]:
xgb(500, 5)

RMSE: 1.6307
R^2 : 0.8700


In [20]:
xgb(400, 5)

RMSE: 1.6447
R^2 : 0.8678


In [21]:
xgb(700, 5)

RMSE: 1.6076
R^2 : 0.8737


In [22]:
# 모듈 불러오기
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(merge_data_x, merge_data_y, test_size=0.3, random_state=42)

# 모델 생성
xgb_reg = xgb.XGBRegressor(
    n_estimators = 500,   # 트리 개수
    learning_rate = 0.05,  # 학습률
    max_depth = 5,        # 트리 최대 깊이
    subsample = 0.8,
    random_state = 42
)
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"R^2 : {r2:.4f}")

RMSE: 1.6732
R^2 : 0.8632


In [23]:
all_y_pred = xgb_reg.predict(merge_data_x)

In [24]:
df_y_result = pd.DataFrame({
    "Prediction": all_y_pred,
    "Actual": merge_data_y
}, index=merge_data_y.index)

In [25]:
sample = df_y_result.sample(n = 20)
display(sample)

,Prediction,Actual
969593,0.406660,0.000000
1141051,0.255088,0.464286
432947,0.619009,1.000000
681119,3.241430,2.571429
510493,0.115368,0.000000
366317,3.518367,3.500000
123104,41.442635,38.363636
24953,3.200289,3.000000
529893,3.480245,7.927273
1024600,0.112335,0.000000


In [26]:
rmse = np.sqrt(mean_squared_error(merge_data_y, all_y_pred))
r2 = r2_score(merge_data_y, all_y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"R^2 : {r2:.4f}")

RMSE: 1.6424
R^2 : 0.8693


In [27]:
xgb_reg.save_model("xgb_model_v3.json")